# 自定义文档加载器
## 概述

基于llm的应用程序通常需要从数据库或文件(如pdf)中提取数据，并将其转换为llm可以使用的格式。在LangChain中，这通常涉及到创建`Document`对象，该对象封装了提取的文本(页面内容)以及包含文档详细信息(如作者姓名或出版日期)的元数据字典。

`Document`对象通常被格式化为输入LLM的提示，允许LLM使用`Document`中的信息来生成所需的响应(例如，总结文档)。文档既可以立即使用，也可以索引到一个向量库中，以便将来检索和使用。

文档加载的主要抽象是

| 名称 | 描述 |
| ---- | ---- |
| Document | 包含`text`和`metadata` |
| BaseLoader | 用于将原始数据转换为`Document` |
| Blob | 二进制数据的一种表示形式，它位于文件或内存中 |
| BaseBlobParser | 解析`Blob`以生成`Document`对象的逻辑 |

本指南将演示如何编写自定义文档加载和文件解析逻辑;具体来说，我们将看到如何
1. 通过从`BaseLoader`子类化创建一个标准文档加载器。
2. 使用`BaseBlobParser`创建解析器，并将其与`Blob`和`blobloader`结合使用。这主要在处理文件时很有用。





### Standard Document Loader

文档加载器可以通过从`BaseLoader`子类化实现，`BaseLoader`提供了加载文档的标准接口。
### Interface
| Method Name | Explanation |
| ---- | ---- |
| lazy_load | 用于逐个惰性加载文档。用于生产代码。 |
| alazy_load | `lazy_load`的异步变体 |
| load | 用于急切地将所有文档装入内存。用于原型设计或交互工作。 |
| aload | 用于急切地将所有文档装入内存。用于原型设计或交互工作。在2024-04年添加到LangChain。 |

- `load`方法是一种方便的方法，仅用于它调用的原型--它只是调用了`list(self.lazy_load())`。
- `alazy_load`有一个默认实现，它将委托给`lazy_load`。如果您使用async，我们建议重写默认实现并提供本机async实现。

### 实施
让我们创建一个标准文档加载器的示例，它加载文件并从文件中的每一行创建一个文档。

In [11]:
from dotenv import load_dotenv, find_dotenv
from langchain.globals import set_debug
import os
load_dotenv(find_dotenv())
set_debug(False)

In [ ]:
from typing import AsyncIterator, Iterator

from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document


class CustomDocumentLoader(BaseLoader):
    """An example document loader that reads a file line by line."""

    def __init__(self, file_path: str) -> None:
        """Initialize the loader with a file path.

        Args:
            file_path: The path to the file to load.
        """
        self.file_path = file_path

    def lazy_load(self) -> Iterator[Document]:  # <-- Does not take any arguments
        """A lazy loader that reads a file line by line.

        When you're implementing lazy load methods, you should use a generator
        to yield documents one by one.
        """
        with open(self.file_path, encoding="utf-8") as f:
            line_number = 0
            for line in f:
                yield Document(
                    page_content=line,
                    metadata={"line_number": line_number, "source": self.file_path},
                )
                line_number += 1

    # alazy_load is OPTIONAL.
    # If you leave out the implementation, a default implementation which delegates to lazy_load will be used!
    async def alazy_load(
        self,
    ) -> AsyncIterator[Document]:  # <-- Does not take any arguments
        """An async lazy loader that reads a file line by line."""
        # Requires aiofiles
        # Install with `pip install aiofiles`
        # https://github.com/Tinche/aiofiles
        import aiofiles

        async with aiofiles.open(self.file_path, encoding="utf-8") as f:
            line_number = 0
            async for line in f:
                yield Document(
                    page_content=line,
                    metadata={"line_number": line_number, "source": self.file_path},
                )
                line_number += 1

In [ ]:
with open("./data/meow.txt", "w", encoding="utf-8") as f:
    quality_content = "meow meow🐱 \n meow meow🐱 \n meow😻😻"
    f.write(quality_content)

loader = CustomDocumentLoader("./data/meow.txt")

In [ ]:
## Test out the lazy load interface
for doc in loader.lazy_load():
    print()
    print(type(doc))
    print(doc)

In [ ]:
## Test out the async implementation
async for doc in loader.alazy_load():
    print()
    print(type(doc))
    print(doc)

In [ ]:
loader.load()

### Working with Files
许多文档加载器都涉及解析文件。这些加载器之间的差异通常源于文件的解析方式，而不是文件的加载方式。例如，您可以使用open来读取PDF或markdown文件的二进制内容，但是需要不同的解析逻辑来将二进制数据转换为文本。

因此，将解析逻辑与加载逻辑解耦是很有帮助的，这使得重用给定解析器变得更容易，而不管数据是如何加载的。

#### BaseBlobParser
`BaseBlobParser`是一个接口，它接受`blob`并输出`Document`对象列表。blob是存储在内存或文件中的数据的表示形式。LangChain python有一个`Blob`原语，它的灵感来自`Blob WebAPI`规范。




In [ ]:
from langchain_core.document_loaders import BaseBlobParser, Blob


class MyParser(BaseBlobParser):
    """A simple parser that creates a document from each line."""

    def lazy_parse(self, blob: Blob) -> Iterator[Document]:
        """Parse a blob into a document line by line."""
        line_number = 0
        with blob.as_bytes_io() as f:
            for line in f:
                line_number += 1
                yield Document(
                    page_content=line,
                    metadata={"line_number": line_number, "source": blob.source},
                )

In [ ]:
blob = Blob.from_path("./data/meow.txt")
parser = MyParser()

In [ ]:
list(parser.lazy_parse(blob))

使用blob API还允许直接从内存中加载内容，而不必从文件中读取内容

In [ ]:
blob = Blob(data=b"some data from memory\nmeow")
list(parser.lazy_parse(blob))

#### Blob
让我们快速浏览一下Blob API。


In [ ]:
blob = Blob.from_path("./data/meow.txt", metadata={"foo": "bar"})
blob.encoding

In [ ]:
blob.as_bytes()

In [ ]:
blob.as_string()

In [ ]:
blob.as_bytes_io()

In [ ]:
blob.metadata

In [ ]:
blob.source

#### Blob Loaders
解析器封装将二进制数据解析为文档所需的逻辑，而blob加载器封装从给定存储位置加载blob所需的逻辑。

目前，LangChain只支持`FileSystemBlobLoader`。       
您可以使用`FileSystemBlobLoader`来加载`blob`，然后使用解析器来解析它们。


In [ ]:
from langchain_community.document_loaders.blob_loaders import FileSystemBlobLoader

blob_loader = FileSystemBlobLoader(path="./data/", glob="*.txt", show_progress=True)

In [ ]:
parser = MyParser()
for blob in blob_loader.yield_blobs():
    for doc in parser.lazy_parse(blob):
        print(doc)
        break

#### Generic Loader
LangChain有一个`GenericLoader`抽象，它由一个`BlobLoader`和一个`BaseBlobParser`组成。

`GenericLoader`旨在提供标准化的类方法，使使用现有`BlobLoader`实现变得容易。目前，只支持`FileSystemBlobLoader`。



In [ ]:
from langchain_community.document_loaders.generic import GenericLoader

loader = GenericLoader.from_filesystem(
    path="./data", glob="*.txt", show_progress=True, parser=MyParser()
)

for idx, doc in enumerate(loader.lazy_load()):
    if idx < 5:
        print(doc)

print("... output truncated for demo purposes")

#### Custom Generic Loader
如果您真的喜欢创建类，您可以创建子类并创建一个类来将逻辑封装在一起。

您可以从这个类创建子类，以使用现有加载器加载内容。

In [ ]:
from typing import Any


class MyCustomLoader(GenericLoader):
    @staticmethod
    def get_parser(**kwargs: Any) -> BaseBlobParser:
        """Override this method to associate a default parser with the class."""
        return MyParser()

In [ ]:
loader = MyCustomLoader.from_filesystem(path="./data", glob="*.txt", show_progress=True)

for idx, doc in enumerate(loader.lazy_load()):
    if idx < 5:
        print(doc)

print("... output truncated for demo purposes")

### CSV
逗号分隔值(CSV)文件是使用逗号分隔值的分隔文本文件。文件的每一行都是一个数据记录。每条记录由一个或多个以逗号分隔的字段组成。



In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path='data/mlb_teams_2012.csv')
data = loader.load()

In [ ]:
data

### Customizing the CSV parsing and loading


In [ ]:
loader = CSVLoader(file_path='./data/mlb_teams_2012.csv', csv_args={
    'delimiter': ',',
    'quotechar': '"',
    'fieldnames': ['MLB Team', 'Payroll in millions', 'Wins']
})

data = loader.load()

In [ ]:
data

### 指定一个列来标识文档源
使用`source`列参数为从每行创建的文档指定源。否则，`file_path`将用作从CSV文件创建的所有文档的源。

当使用从CSV文件加载的文档用于使用源回答问题的链时，这很有用。



In [ ]:
loader = CSVLoader(file_path='./data/mlb_teams_2012.csv', source_column="Team")

data = loader.load()

In [ ]:
data

### 文件目录
这将介绍如何加载目录中的所有文档。

在底层，默认情况下使用`UnstructuredLoader`。

In [ ]:
from langchain_community.document_loaders import DirectoryLoader


我们可以使用`glob`参数来控制加载哪些文件。注意，这里没有加载`.rst`文件或`.html`文件。

In [ ]:
loader = DirectoryLoader('./data', glob="**/*.txt")

In [ ]:
docs = loader.load()
docs

In [ ]:
len(docs)

### 显示进度条
默认情况下不会显示进度条。要显示进度条，请安装`tqdm`库(例如`pip install tqdm`)，并将`show progress`参数设置为`True`。

In [ ]:
loader = DirectoryLoader('./data', glob="**/*.txt", show_progress=True)
docs = loader.load()

### Use multithreading
默认情况下，加载发生在一个线程中。为了使用多个线程，将use多线程标志设置为true。

In [ ]:
loader = DirectoryLoader('./data', glob="**/*.md", use_multithreading=True)
docs = loader.load()

### 更改装入器类
默认情况下，它使用`UnstructuredLoader`类。但是，您可以很容易地更改加载器的类型。

In [ ]:
from langchain_community.document_loaders import TextLoader
loader = DirectoryLoader('./data', glob="**/*.txt", loader_cls=TextLoader)
docs = loader.load()
len(docs)

如果需要加载`Python`源代码文件，请使用`PythonLoader`。

In [ ]:
from langchain_community.document_loaders import PythonLoader
loader = DirectoryLoader('./data', glob="**/*.py", loader_cls=PythonLoader)
docs = loader.load()
len(docs)

### 自动检测文件编码与TextLoader
在这个例子中，我们将看到一些策略，这些策略在使用`TextLoader`类从目录加载大量任意文件时非常有用。
首先为了说明这个问题，让我们尝试用任意编码加载多个文本。

In [ ]:
path = './data'
loader = DirectoryLoader(path, glob="**/*.txt", loader_cls=TextLoader)
loader.load()

文件`example-non-utf8.txt`使用不同的编码，因此`load()`函数失败时将显示有用的消息，指出哪个文件解码失败。

`TextLoader`的默认行为，任何失败加载任何文档将失败的整个加载过程，没有文档被加载。

### B. Silent fail

我们可以将参数`silent errors`传递给`DirectoryLoader`，以跳过无法加载的文件并继续加载过程。

In [ ]:
loader = DirectoryLoader(path, glob="**/*.txt", loader_cls=TextLoader, silent_errors=True)
docs = loader.load()

doc_sources = [doc.metadata['source']  for doc in docs]
doc_sources

### C. Auto detect encodings
我们也可以通过将自动检测编码传递给加载器类来要求`TextLoader`在失败之前自动检测文件编码。

In [ ]:
text_loader_kwargs={'autodetect_encoding': True}
loader = DirectoryLoader(path, glob="**/*.txt", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
docs = loader.load()

doc_sources = [doc.metadata['source']  for doc in docs]
doc_sources

### HTML
超文本标记语言或HTML是设计用于在web浏览器中显示的文档的标准标记语言。

这将介绍如何将HTML文档加载为我们可以在后续使用的文档格式。

In [ ]:
from langchain_community.document_loaders import UnstructuredHTMLLoader
loader = UnstructuredHTMLLoader("data/fake-content.html")
data = loader.load()
data

### Loading HTML with BeautifulSoup4
我们还可以使用`BeautifulSoup4`来使用`BSHTMLLoader`加载HTML文档。这将从HTML中将文本提取为页面内容，并将页面标题作为标题提取为元数据。

In [ ]:
from langchain_community.document_loaders import BSHTMLLoader
loader = BSHTMLLoader("data/fake-content.html")
data = loader.load()
data

#### Loading HTML with SpiderLoader
蜘蛛是最快的爬行动物。它将任何网站转换为纯`HTML`, `markdown`，元数据或文本，同时使您能够使用人工智能进行自定义操作。

Spider允许您使用高性能代理来防止检测，缓存AI动作，webhook用于爬行状态，计划爬行等…
##### Prerequisite
您需要有一个Spider api密钥才能使用这个加载器。你可以在spider.cloud上买到。



In [ ]:
from langchain_community.document_loaders.spider import SpiderLoader

loader = SpiderLoader(
    api_key=os.environ["SPIDER_API_KEY"], 
    url="https://spider.cloud", 
    mode="crawl"
)

data = loader.load()

### Loading HTML with FireCrawlLoader
FireCrawl抓取并转换任何网站为降价。它会抓取所有可访问的子页面，并为每个页面提供清晰的标记和元数据。

firerawl处理复杂的任务，如反向代理、缓存、速率限制和JavaScript阻止的内容。

**Prerequisite**      
您需要有一个firerawl API密钥才能使用这个加载程序。你可以通过在firerawl注册获得一个。

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community firecrawl-py

from langchain_community.document_loaders import FireCrawlLoader


loader = FireCrawlLoader(
    api_key="YOUR_API_KEY", url="https://firecrawl.dev", mode="crawl"
)

data = loader.load()

### Loading HTML with AzureAIDocumentIntelligenceLoader

Azure AI文档智能（以前称为Azure形式识别器）是基于机器学习的服务，它可以提取文本（包括手写），表格，文档结构（例如，标题，截面标题等）和数字或键值配对扫描PDF，图像，办公室和HTML文件。文档智能支持PDF，JPEG/JPG，PNG，BMP，TIFF，HEIF，DOCX，XLSX，PPTX和HTML。

当前使用文档智能的加载器实现可以按页面合并内容并将其转换为LangChain文档。默认的输出格式是markdown，它可以很容易地与MarkdownHeaderTextSplitter链接，用于语义文档分块。还可以使用mode="single"或mode="page"返回单个页面或按页面分割的文档中的纯文本。

In [ ]:
!pip install --upgrade --quiet  langchain langchain-community azure-ai-documentintelligence

from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader

file_path = "<filepath>"
endpoint = "<endpoint>"
key = "<key>"
loader = AzureAIDocumentIntelligenceLoader(
    api_endpoint=endpoint, api_key=key, file_path=file_path, api_model="prebuilt-layout"
)

documents = loader.load()

## JSON
JSON (JavaScript Object Notation)是一种开放的标准文件格式和数据交换格式，它使用人类可读的文本来存储和传输由属性值对和数组(或其他可序列化的值)组成的数据对象。

JSON行是一种文件格式，其中每行都是一个有效的JSON值。

JSONLoader使用指定的jq模式来解析JSON文件。它使用jq python包。有关jq语法的详细文档，请参阅本手册。



In [15]:
from langchain_community.document_loaders import JSONLoader
import json
from pathlib import Path
from pprint import pprint


file_path='./data/facebook_chat.json'
data = json.loads(Path(file_path).read_text())
data

{'participants': [{'name': 'User 1'}, {'name': 'User 2'}],
 'messages': [{'sender_name': 'User 2',
   'timestamp_ms': 1675597571851,
   'content': 'Bye!'},
  {'sender_name': 'User 1',
   'timestamp_ms': 1675597435669,
   'content': 'Oh no worries! Bye'},
  {'sender_name': 'User 2',
   'timestamp_ms': 1675596277579,
   'content': 'No Im sorry it was my mistake, the blue one is not for sale'},
  {'sender_name': 'User 1',
   'timestamp_ms': 1675595140251,
   'content': 'I thought you were selling the blue one!'},
  {'sender_name': 'User 1',
   'timestamp_ms': 1675595109305,
   'content': 'Im not interested in this bag. Im interested in the blue one!'},
  {'sender_name': 'User 2',
   'timestamp_ms': 1675595068468,
   'content': 'Here is $129'},
  {'sender_name': 'User 2',
   'timestamp_ms': 1675595060730,
   'photos': [{'uri': 'url_of_some_picture.jpg',
     'creation_timestamp': 1675595059}]},
  {'sender_name': 'User 2',
   'timestamp_ms': 1675595045152,
   'content': 'Online is at least 

### Using JSONLoader

假设我们有兴趣提取JSON数据的消息键中的内容字段下的值。这可以通过JSONLoader轻松完成，如下所示。
#### JSON file


In [ ]:
loader = JSONLoader(
    file_path='./data/facebook_chat.json',
    jq_schema='.messages[].content',
    text_content=False)

data = loader.load()
data

#### JSON Lines file
如果您想从JSON Lines文件加载文档，则传递`JSON Lines =True`并指定jq模式以从单个JSON对象提取页面内容。

In [ ]:
file_path = './data/facebook_chat_messages.jsonl'
pprint(Path(file_path).read_text())
loader = JSONLoader(
    file_path='./data/facebook_chat_messages.jsonl',
    jq_schema='.content',
    text_content=False,
    json_lines=True)

data = loader.load()
data

另一个选项是`set jq schema='.'`并提供`content_key`


In [ ]:
loader = JSONLoader(
    file_path='./data/facebook_chat_messages.jsonl',
    jq_schema='.',
    content_key='sender_name',
    json_lines=True)

data = loader.load()
data

### JSON file with jq schema content_key
要使用jq模式中的内容键从JSON文件加载文档，请设置内容键jq parsable=True。确保内容键是兼容的，并且可以使用jq模式进行解析。

In [ ]:
file_path = './data/sample.json'
pprint(Path(file_path).read_text())
loader = JSONLoader(
    file_path=file_path,
    jq_schema=".data[]",
    content_key=".attributes.message",
    is_content_key_jq_parsable=True,
)

data = loader.load()

### Extracting metadata
通常，我们希望将JSON文件中可用的元数据包含到从内容创建的文档中。
下面演示如何使用`JSONLoader`提取元数据。

有一些关键的变化需要注意。在前面的示例中，我们没有收集元数据，我们设法在模式中直接指定可以从中提取`page_content`的值。

`.messages[].content`

在当前的示例中，我们必须告诉加载器遍历消息字段中的记录。jq模式必须是


这允许我们将记录(dict)传递到必须实现的元数据函数中。元数据函数负责确定记录中的哪些信息片段应该包含在存储在最终Document对象中的元数据中。

此外，我们现在必须通过`content_key`参数在加载器中显式指定需要从中提取页面内容值的记录中的键。



In [2]:
# Define the metadata extraction function.
from langchain_community.document_loaders import JSONLoader
def metadata_func(record: dict, metadata: dict) -> dict:

    metadata["sender_name"] = record.get("sender_name")
    metadata["timestamp_ms"] = record.get("timestamp_ms")

    return metadata


loader = JSONLoader(
    file_path='./data/facebook_chat.json',
    jq_schema='.messages[]',
    content_key="content",
    metadata_func=metadata_func
)

data = loader.load()
data

---- {'sender_name': 'User 2', 'timestamp_ms': 1675597571851, 'content': 'Bye!'}
---- {'sender_name': 'User 1', 'timestamp_ms': 1675597435669, 'content': 'Oh no worries! Bye'}
---- {'sender_name': 'User 2', 'timestamp_ms': 1675596277579, 'content': 'No Im sorry it was my mistake, the blue one is not for sale'}
---- {'sender_name': 'User 1', 'timestamp_ms': 1675595140251, 'content': 'I thought you were selling the blue one!'}
---- {'sender_name': 'User 1', 'timestamp_ms': 1675595109305, 'content': 'Im not interested in this bag. Im interested in the blue one!'}
---- {'sender_name': 'User 2', 'timestamp_ms': 1675595068468, 'content': 'Here is $129'}
---- {'sender_name': 'User 2', 'timestamp_ms': 1675595060730, 'content': 'Online is at least $100', 'photos': [{'uri': 'url_of_some_picture.jpg', 'creation_timestamp': 1675595059}]}
---- {'sender_name': 'User 2', 'timestamp_ms': 1675595045152, 'content': 'Online is at least $100'}
---- {'sender_name': 'User 1', 'timestamp_ms': 1675594799696, 

[Document(page_content='Bye!', metadata={'source': '/Users/dyz/Documents/DL/10-langchain/data/facebook_chat.json', 'seq_num': 1, 'sender_name': 'User 2', 'timestamp_ms': 1675597571851}),
 Document(page_content='Oh no worries! Bye', metadata={'source': '/Users/dyz/Documents/DL/10-langchain/data/facebook_chat.json', 'seq_num': 2, 'sender_name': 'User 1', 'timestamp_ms': 1675597435669}),
 Document(page_content='No Im sorry it was my mistake, the blue one is not for sale', metadata={'source': '/Users/dyz/Documents/DL/10-langchain/data/facebook_chat.json', 'seq_num': 3, 'sender_name': 'User 2', 'timestamp_ms': 1675596277579}),
 Document(page_content='I thought you were selling the blue one!', metadata={'source': '/Users/dyz/Documents/DL/10-langchain/data/facebook_chat.json', 'seq_num': 4, 'sender_name': 'User 1', 'timestamp_ms': 1675595140251}),
 Document(page_content='Im not interested in this bag. Im interested in the blue one!', metadata={'source': '/Users/dyz/Documents/DL/10-langchain/d

### The metadata_func
如上所示，元数据函数接受JSONLoader生成的默认元数据。这允许用户完全控制元数据的格式。

例如，默认元数据包含源键和seq num键。但是，JSON数据也可能包含这些键。然后，用户可以利用元数据函数重命名默认键，并使用JSON数据中的键。

下面的示例展示了如何修改源代码，使其只包含相对于langchain目录的文件源信息。

In [3]:
# Define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:

    metadata["sender_name"] = record.get("sender_name")
    metadata["timestamp_ms"] = record.get("timestamp_ms")

    if "source" in metadata:
        source = metadata["source"].split("/")
        source = source[source.index("langchain"):]
        metadata["source"] = "/".join(source)

    return metadata


loader = JSONLoader(
    file_path='./data/facebook_chat.json',
    jq_schema='.messages[]',
    content_key="content",
    metadata_func=metadata_func
)

data = loader.load()

---- {'sender_name': 'User 2', 'timestamp_ms': 1675597571851, 'content': 'Bye!'}


ValueError: 'langchain' is not in list

### Common JSON structures with jq schema
下面的列表提供了一个可能的jq模式的参考，用户可以根据结构从JSON数据中提取内容。
```json

JSON        -> [{"text": ...}, {"text": ...}, {"text": ...}]
jq_schema   -> ".[].text"

JSON        -> {"key": [{"text": ...}, {"text": ...}, {"text": ...}]}
jq_schema   -> ".key[].text"

JSON        -> ["...", "...", "..."]
jq_schema   -> ".[]"
```

### Markdown
Markdown是一种轻量级标记语言，用于使用纯文本编辑器创建格式化文本。
这将介绍如何将Markdown文档加载为我们可以在后续使用的文档格式。

!pip install unstructured > /dev/null

In [6]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
markdown_path = "./../README.md"
loader = UnstructuredMarkdownLoader(markdown_path)
data = loader.load()
data


[Document(page_content='学习笔记\n\n01 Python\n\n02 数据分析\n\n03 深度学习框架\n\nPytorch\n\nBatchNorm\n\n04 基础算法\n\n05 Statistical Inference\n\n重要性采样\n\n逆变换采样\n\n拒绝抽样\n\n吉布斯采样\n\n朗格万-蒙特卡洛\n\nMetropolis Hastings\n\n06 机器学习scikit-learn\n\nLinerRegression\n\nLogistics Regression\n\nTree\n\nRandomForest\n\nGBDT\n\nXGBoost\n\n深度学习模型\n\n注意力机制\n\n非参数注意力\n\n参数注意力\n\n内积注意力\n\n加性注意力\n\n多头注意力机制\n\nself-Attention\n\nPostions位置编码\n\nAIGC\n\nVAE\n\nGAN\n\nChatGPT', metadata={'source': './../README.md'})]

##### Retain Elements
在底层，非结构化为不同的文本块创建不同的“元素”。默认情况下，我们将它们组合在一起，但您可以通过指定`mode="elements"`轻松保持这种分离。

In [7]:
loader = UnstructuredMarkdownLoader(markdown_path, mode="elements")
data = loader.load()
data

[Document(page_content='学习笔记', metadata={'source': './../README.md', 'last_modified': '2024-05-02T20:56:44', 'page_number': 1, 'languages': ['eng'], 'filetype': 'text/markdown', 'file_directory': './..', 'filename': 'README.md', 'category': 'Title'}),
 Document(page_content='01 Python', metadata={'source': './../README.md', 'last_modified': '2024-05-02T20:56:44', 'page_number': 1, 'languages': ['eng'], 'filetype': 'text/markdown', 'file_directory': './..', 'filename': 'README.md', 'category': 'Title'}),
 Document(page_content='02 数据分析', metadata={'source': './../README.md', 'last_modified': '2024-05-02T20:56:44', 'page_number': 1, 'languages': ['eng'], 'filetype': 'text/markdown', 'file_directory': './..', 'filename': 'README.md', 'category': 'Title'}),
 Document(page_content='03 深度学习框架', metadata={'source': './../README.md', 'last_modified': '2024-05-02T20:56:44', 'page_number': 1, 'languages': ['eng'], 'filetype': 'text/markdown', 'file_directory': './..', 'filename': 'README.md', 'c

## Microsoft Office
Microsoft Office办公软件套件包括Microsoft Word、Microsoft Excel、Microsoft PowerPoint、Microsoft Outlook和Microsoft OneNote。适用于Microsoft Windows和macOS操作系统。它也可以在安卓和iOS上使用。

这将介绍如何将常用的文件格式(包括DOCX、XLSX和PPTX文档)加载为我们可以在下游使用的文档格式。

### Loading DOCX, XLSX, PPTX with AzureAIDocumentIntelligenceLoader
Azure AI文档智能（以前称为Azure形式识别器）是基于机器学习的服务，它可以提取文本（包括手写），表格，文档结构（例如，标题，截面标题等）和数字或键值配对扫描PDF，图像，办公室和HTML文件。文档智能支持`PDF`，`JPEG/JPG`，`PNG`，`BMP`，`TIFF`，`HEIF`，`DOCX`，`XLSX`，`PPTX`和`HTML`。

当前使用文档智能的加载器实现可以按页面合并内容并将其转换为`LangChain`文档。默认的输出格式是`markdown`，它可以很容易地与`MarkdownHeaderTextSplitter`链接，用于语义文档分块。还可以使用`mode="single"`或`mode="page"`返回单个页面或按页面分割的文档中的纯文本。

**前提**
在3个预览区之一的Azure AI文档智能资源:美国东部，美国西部，西欧-如果你没有，请按照本文档创建一个。您将把<endpoint>和<key>作为参数传递给加载器。





In [ ]:
!pip install --upgrade --quiet  langchain langchain-community azure-ai-documentintelligence

from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader

file_path = "<filepath>"
endpoint = "<endpoint>"
key = "<key>"
loader = AzureAIDocumentIntelligenceLoader(
    api_endpoint=endpoint, api_key=key, file_path=file_path, api_model="prebuilt-layout"
)

documents = loader.load()

### PDF
可移植文档格式(PDF)，标准化为ISO 32000，是Adobe在1992年开发的一种文件格式，用于以独立于应用软件、硬件和操作系统的方式呈现文档，包括文本格式和图像。

这将介绍如何将PDF文档加载为我们在下游使用的Document格式。

#### Using PyPDF
使用pypdf将PDF加载到文档数组中，其中每个文档包含页面内容和带有页码的元数据。




In [8]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./data/layout-parser-paper.pdf")
pages = loader.load_and_split()

In [9]:
pages[0]

Document(page_content='LayoutParser : A Uniﬁed Toolkit for Deep\nLearning Based Document Image Analysis\nZejiang Shen1( \x00), Ruochen Zhang2, Melissa Dell3, Benjamin Charles Germain\nLee4, Jacob Carlson3, and Weining Li5\n1Allen Institute for AI\nshannons@allenai.org\n2Brown University\nruochen zhang@brown.edu\n3Harvard University\n{melissadell,jacob carlson }@fas.harvard.edu\n4University of Washington\nbcgl@cs.washington.edu\n5University of Waterloo\nw422li@uwaterloo.ca\nAbstract. Recent advances in document image analysis (DIA) have been\nprimarily driven by the application of neural networks. Ideally, research\noutcomes could be easily deployed in production and extended for further\ninvestigation. However, various factors like loosely organized codebases\nand sophisticated model conﬁgurations complicate the easy reuse of im-\nportant innovations by a wide audience. Though there have been on-going\neﬀorts to improve reusability and simplify deep learning (DL) model\ndevelopment in 

In [12]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())
docs = faiss_index.similarity_search("How will the community be engaged?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

9: 10 Z. Shen et al.
Fig. 4: Illustration of (a) the original historical Japanese document with layout
detection results and (b) a recreated version of the document image that achieves
much better character recognition recall. The reorganization algorithm rearranges
the tokens based on the their detect
3: 4 Z. Shen et al.
Efficient Data AnnotationC u s t o m i z e d  M o d e l  T r a i n i n gModel Cust omizationDI A Model HubDI A Pipeline SharingCommunity PlatformLa y out Detection ModelsDocument Images 
T h e  C o r e  L a y o u t P a r s e r  L i b r a r yOCR ModuleSt or age & VisualizationLa y ou


#### Extracting images
使用`rapidocr-onnxruntime`包，我们也可以提取图像作为文本


In [13]:
loader = PyPDFLoader("https://arxiv.org/pdf/2103.15348.pdf", extract_images=True)
pages = loader.load()
pages[4].page_content

'LayoutParser : A Uniﬁed Toolkit for DL-Based DIA 5\nTable 1: Current layout detection models in the LayoutParser model zoo\nDataset Base Model1Large Model Notes\nPubLayNet [38] F / M M Layouts of modern scientiﬁc documents\nPRImA [3] M - Layouts of scanned modern magazines and scientiﬁc reports\nNewspaper [17] F - Layouts of scanned US newspapers from the 20th century\nTableBank [18] F F Table region on modern scientiﬁc and business document\nHJDataset [31] F / M - Layouts of history Japanese documents\n1For each dataset, we train several models of diﬀerent sizes for diﬀerent needs (the trade-oﬀ between accuracy\nvs. computational cost). For “base model” and “large model”, we refer to using the ResNet 50 or ResNet 101\nbackbones [ 13], respectively. One can train models of diﬀerent architectures, like Faster R-CNN [ 28] (F) and Mask\nR-CNN [ 12] (M). For example, an F in the Large Model column indicates it has a Faster R-CNN model trained\nusing the ResNet 101 backbone. The platform i

#### Using PyMuPDF
这是最快的PDF解析选项，包含关于PDF及其页面的详细元数据，并且每页返回一个文档。


In [16]:
from langchain_community.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("./data/layout-parser-paper.pdf")
data = loader.load()
data[0]

Document(page_content='LayoutParser: A Uniﬁed Toolkit for Deep\nLearning Based Document Image Analysis\nZejiang Shen1 (\x00), Ruochen Zhang2, Melissa Dell3, Benjamin Charles Germain\nLee4, Jacob Carlson3, and Weining Li5\n1 Allen Institute for AI\nshannons@allenai.org\n2 Brown University\nruochen zhang@brown.edu\n3 Harvard University\n{melissadell,jacob carlson}@fas.harvard.edu\n4 University of Washington\nbcgl@cs.washington.edu\n5 University of Waterloo\nw422li@uwaterloo.ca\nAbstract. Recent advances in document image analysis (DIA) have been\nprimarily driven by the application of neural networks. Ideally, research\noutcomes could be easily deployed in production and extended for further\ninvestigation. However, various factors like loosely organized codebases\nand sophisticated model conﬁgurations complicate the easy reuse of im-\nportant innovations by a wide audience. Though there have been on-going\neﬀorts to improve reusability and simplify deep learning (DL) model\ndevelopment 

此外，您可以将PyMuPDF文档中的任何选项作为关键字参数传递给load调用，并将其传递给`get_text()`调用。

#### Using MathPix
受丹尼尔·格罗斯（Daniel Gross）https://gist.github.com/danielgross/3ab4104e14faccc12b49200843adab21的启发


In [17]:
from langchain_community.document_loaders import MathpixPDFLoader
loader = MathpixPDFLoader("data/layout-parser-paper.pdf")
data = loader.load()
data


ValueError: Did not find mathpix_api_key, please add an environment variable `MATHPIX_API_KEY` which contains it, or pass `mathpix_api_key` as a named parameter.

### Using Unstructured
unstructured[all-docs]包目前支持加载文本文件、ppt、html、pdf、图像等。

In [22]:
from langchain_community.document_loaders import UnstructuredPDFLoader
loader = UnstructuredPDFLoader("data/layout-parser-paper.pdf")

In [23]:
data = loader.load()
data[0]

ImportError: cannot import name 'open_filename' from 'pdfminer.utils' (/Users/dyz/opt/anaconda3/envs/langchain/lib/python3.10/site-packages/pdfminer/utils.py)

#### Retain Elements
在底层，非结构化为不同的文本块创建不同的“元素”。默认情况下，我们将它们组合在一起，但您可以通过指定`mode="elements"`轻松保持这种分离。



In [24]:
loader = UnstructuredPDFLoader("data/layout-parser-paper.pdf", mode="elements")
data = loader.load()
data[0]

#### Fetching remote PDFs using Unstructured
这涵盖了如何将在线PDF加载到可以使用下游的文档格式中。这可以用于各种在线PDF网站，例如https://open.umn.edu/opentextbooks/textbooks/和https://arxiv.org/archive/

注意:所有其他PDF加载器也可以用来获取远程PDF，但是`OnlinePDFLoader`是一个遗留函数，并且专门与`UnstructuredPDFLoader`一起工作。

In [ ]:
from langchain_community.document_loaders import OnlinePDFLoader
loader = OnlinePDFLoader("https://arxiv.org/pdf/2302.03803.pdf")
data = loader.load()
data[0]

### Using PyPDFium2


In [26]:
from langchain_community.document_loaders import PyPDFium2Loader
loader = PyPDFium2Loader("data/layout-parser-paper.pdf")
data = loader.load()
data

/Users/dyz/opt/anaconda3/envs/langchain/lib/python3.10/site-packages/pypdfium2/_helpers/textpage.py:81: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


[Document(page_content='LayoutParser: A Unified Toolkit for Deep\r\nLearning Based Document Image Analysis\r\nZejiang Shen\r\n1\r\n(), Ruochen Zhang\r\n2\r\n, Melissa Dell\r\n3\r\n, Benjamin Charles Germain\r\nLee\r\n4\r\n, Jacob Carlson\r\n3\r\n, and Weining Li\r\n5\r\n1 Allen Institute for AI\r\nshannons@allenai.org 2 Brown University\r\nruochen zhang@brown.edu 3 Harvard University\r\n{melissadell,jacob carlson}@fas.harvard.edu\r\n4 University of Washington\r\nbcgl@cs.washington.edu 5 University of Waterloo\r\nw422li@uwaterloo.ca\r\nAbstract. Recent advances in document image analysis (DIA) have been\r\nprimarily driven by the application of neural networks. Ideally, research\r\noutcomes could be easily deployed in production and extended for further\r\ninvestigation. However, various factors like loosely organized codebases\r\nand sophisticated model configurations complicate the easy reuse of im\x02portant innovations by a wide audience. Though there have been on-going\r\nefforts t

### Using PDFMiner
PDFMiner是一个可以帮助从PDF文档中提取信息和分析数据的工具。     
!pip install pdfminer.six

In [30]:
from langchain_community.document_loaders import PDFMinerLoader
loader = PDFMinerLoader("data/layout-parser-paper.pdf")
data = loader.load()
data[0]

ImportError: `pdfminer` package not found, please install it with `pip install pdfminer.six`

#### Using PDFMiner to generate HTML text
这可能有助于将文本分解为分段，因为可以通过`BeautifulSoup`解析输出HTML内容，以获取有关字体大小，页码，PDF标题/页脚等的更结构化和丰富的信息。



In [31]:
from langchain_community.document_loaders import PDFMinerPDFasHTMLLoader
loader = PDFMinerPDFasHTMLLoader("data/layout-parser-paper.pdf")
data = loader.load()[0]   # entire PDF is loaded as a single Document


ImportError: `pdfminer` package not found, please install it with `pip install pdfminer.six`

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(data.page_content,'html.parser')
content = soup.find_all('div')

import re
cur_fs = None
cur_text = ''
snippets = []   # first collect all snippets that have the same font size
for c in content:
    sp = c.find('span')
    if not sp:
        continue
    st = sp.get('style')
    if not st:
        continue
    fs = re.findall('font-size:(\d+)px',st)
    if not fs:
        continue
    fs = int(fs[0])
    if not cur_fs:
        cur_fs = fs
    if fs == cur_fs:
        cur_text += c.text
    else:
        snippets.append((cur_text,cur_fs))
        cur_fs = fs
        cur_text = c.text
snippets.append((cur_text,cur_fs))
# Note: The above logic is very straightforward. One can also add more strategies such as removing duplicate snippets (as
# headers/footers in a PDF appear on multiple pages so if we find duplicates it's safe to assume that it is redundant info)

In [ ]:
from langchain_community.docstore.document import Document
cur_idx = -1
semantic_snippets = []
# Assumption: headings have higher font size than their respective content
for s in snippets:
    # if current snippet's font size > previous section's heading => it is a new heading
    if not semantic_snippets or s[1] > semantic_snippets[cur_idx].metadata['heading_font']:
        metadata={'heading':s[0], 'content_font': 0, 'heading_font': s[1]}
        metadata.update(data.metadata)
        semantic_snippets.append(Document(page_content='',metadata=metadata))
        cur_idx += 1
        continue

    # if current snippet's font size <= previous section's content => content belongs to the same section (one can also create
    # a tree like structure for sub sections if needed but that may require some more thinking and may be data specific)
    if not semantic_snippets[cur_idx].metadata['content_font'] or s[1] <= semantic_snippets[cur_idx].metadata['content_font']:
        semantic_snippets[cur_idx].page_content += s[0]
        semantic_snippets[cur_idx].metadata['content_font'] = max(s[1], semantic_snippets[cur_idx].metadata['content_font'])
        continue

    # if current snippet's font size > previous section's content but less than previous section's heading than also make a new
    # section (e.g. title of a PDF will have the highest font size but we don't want it to subsume all sections)
    metadata={'heading':s[0], 'content_font': 0, 'heading_font': s[1]}
    metadata.update(data.metadata)
    semantic_snippets.append(Document(page_content='',metadata=metadata))
    cur_idx += 1

### PyPDF Directory


In [32]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("./data")
docs = loader.load()

In [33]:
docs

[Document(page_content='LayoutParser : A Uniﬁed Toolkit for Deep\nLearning Based Document Image Analysis\nZejiang Shen1( \x00), Ruochen Zhang2, Melissa Dell3, Benjamin Charles Germain\nLee4, Jacob Carlson3, and Weining Li5\n1Allen Institute for AI\nshannons@allenai.org\n2Brown University\nruochen zhang@brown.edu\n3Harvard University\n{melissadell,jacob carlson }@fas.harvard.edu\n4University of Washington\nbcgl@cs.washington.edu\n5University of Waterloo\nw422li@uwaterloo.ca\nAbstract. Recent advances in document image analysis (DIA) have been\nprimarily driven by the application of neural networks. Ideally, research\noutcomes could be easily deployed in production and extended for further\ninvestigation. However, various factors like loosely organized codebases\nand sophisticated model conﬁgurations complicate the easy reuse of im-\nportant innovations by a wide audience. Though there have been on-going\neﬀorts to improve reusability and simplify deep learning (DL) model\ndevelopment in

### Using PDFPlumbe
与PyMuPDF一样，输出文档包含关于PDF及其页面的详细元数据，并且每页返回一个文档。



In [41]:
from langchain_community.document_loaders import PDFPlumberLoader
loader = PDFPlumberLoader("data/layout-parser-paper.pdf")
data = loader.load()
data[0]

TypeError: 'type' object is not subscriptable

### Using AmazonTextractPDFParser
AmazonTextractPDFLoader调用Amazon文本服务将pdf转换为文档结构。加载器目前使用的是纯OCR，根据需求，还计划了更多的功能，比如布局支持。支持单页和多页文档，最多可达3000页，大小为512 MB。

为了使调用成功，需要一个AWS帐户，类似于AWS CLI的要求。

除了AWS配置之外，它与其他PDF加载器非常相似，同时也支持JPEG、PNG、TIFF和非本机PDF格式。

In [42]:
from langchain_community.document_loaders import AmazonTextractPDFLoader
loader = AmazonTextractPDFLoader("data/alejandro_rosalez_sample-small.jpeg")
documents = loader.load()
documents

ValueError: File path data/alejandro_rosalez_sample-small.jpeg is not a valid file or url

### Using AzureAIDocumentIntelligenceLoader
Azure AI文档智能（以前称为Azure形式识别器）是基于机器学习的服务，它可以提取文本（包括手写），表格，文档结构（例如，标题，截面标题等）和数字或键值配对扫描PDF，图像，办公室和HTML文件。文档智能支持PDF，JPEG/JPG，PNG，BMP，TIFF，HEIF，DOCX，XLSX，PPTX和HTML。

当前使用文档智能的加载器实现可以按页面合并内容并将其转换为LangChain文档。默认的输出格式是markdown，它可以很容易地与`MarkdownHeaderTextSplitter`链接，用于语义文档分块。还可以使用mode="single"或mode="page"返回单个页面或按页面分割的文档中的纯文本。

在3个预览区之一的Azure AI文档智能资源:美国东部，美国西部，西欧-如果你没有，请按照本文档创建一个。您将把<endpoint>和<key>作为参数传递给加载器。



In [ ]:
!pip install --upgrade --quiet  langchain langchain-community azure-ai-documentintelligence

from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader

file_path = "<filepath>"
endpoint = "<endpoint>"
key = "<key>"
loader = AzureAIDocumentIntelligenceLoader(
    api_endpoint=endpoint, api_key=key, file_path=file_path, api_model="prebuilt-layout"
)

documents = loader.load()

### Using UpstageLayoutAnalysisLoader
`Upstagelayoutanalysis Loader`调用了`UpStage Layout Analysis API`来检测各种文档格式的文档元素，包括表和数字。该装载机采用纯OCR提取文本信息并检测诸如`JPEG`，`PNG`，`BMP`，`PDF`，`TIFF`和`HEIC`文件之类的文档中的元素。对于数字诞生的PDF文档，用户可以通过设置`use_ocr = false`（这是默认值）来放弃OCR并在文件中使用文本信息。支持单页和多页文档，当`use_ocr = true`时，限制为100页，文件大小为50 mb，而当`use_ocr = false`（仅适用于PDF文件）时，没有限制。

要访问后台布局分析API，您需要API访问令牌。请参考提供的快速入门指南，以获得访问令牌并开始使用后台布局分析API。

In [ ]:
import os

os.environ["UPSTAGE_DOCUMENT_AI_API_KEY"] = "YOUR_API_KEY"

from langchain_upstage import UpstageLayoutAnalysisLoader

file_path = "/PATH/TO/FILE.pdf"

loader = UpstageLayoutAnalysisLoader(file_path)
data = loader.load()